In [ ]:
import os

# Prompt for the API key
api_key = input("Enter your OpenAI API key: ")

# Set the environment variable
os.environ["OPENAI_API_KEY"] = api_key
from openai import OpenAI

client = OpenAI()

In [ ]:
import os
import openai

# Directory containing the files
directory_path = './content/drlee'

# List all files in the directory
all_file_names = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Initialize OpenAI client (ensure you have set your API key)
client = openai.OpenAI()

# Upload files and collect file IDs
file_ids = []
for file_name in all_file_names:
    with open(file_name, 'rb') as file:
        response = client.files.create(file=file, purpose='assistants')
        file_ids.append(response.id)  # Access the id attribute

# Now file_ids contains the IDs of all uploaded files

In [ ]:
# Create the Assistant with both Retrieval and Code Interpreter tools
assistant = client.beta.assistants.create(
    name="Academic Co-Pilot",
    instructions="You are an AI assistant tailored to a Professor's expertise in [Subject Area]: AI and Data Analytics. Provide detailed and accurate information based on the provided resources ONLY.  If there is no context in the resources then provide a blank response.  Please provide citations for the documents and page numbers",
    tools=[
        {"type": "retrieval"},
        {"type": "code_interpreter"}
    ],
    model="gpt-4-1106-preview",
    file_ids=file_ids  # Assuming 'file_ids' is a list of IDs from previously uploaded files
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
user_query = "What are stop words?"
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_query
)
print(message)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions='''Please address the user as MDC Student X. If there is no context for the answer in the documents then don't return anything'''
)

In [ ]:
import time

while True:
    # Retrieve the run's current status
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run)

    # Check if the run has a 'completed_at' value
    if run.completed_at is not None:
        break

    # Wait for 7 seconds before checking again
    time.sleep(7)

# Continue with the next steps after the loop exits

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages)

In [ ]:
import textwrap

# Formatting and displaying the results
for message in messages.data:
    # Extracting the message content
    if message.content and message.content[0].text.value.strip():
        content = message.content[0].text.value
    else:
        content = "There is no context for this question."

    # Wrap text to the width of the colab cell
    wrapped_content = textwrap.fill(content, width=80)
    
    # Printing the role and content in a readable format
    print(f"Role: {message.role}")
    print("Content:")
    print(wrapped_content)
    print("-" * 80)  # Separator for readability